A ChatBot is created using NLP and Deep Learning, to guide students to see which universities are offering admissions as per their secured marks.

## Installing Required Libraries

In [1]:
!pip install --upgrade pip

In [2]:
!pip install --upgrade tensorflow

In [3]:
# pip install --upgrade tensorflow

In [7]:
!pip3 install --user --upgrade tensorflow

In [8]:
print(tf.reduce_sum(tf.random.normal([100, 100])))

NameError: name 'tf' is not defined

In [ ]:
!pip install --user -U nltk

In [ ]:
!pip install -U nltk

In [ ]:
!pip install nltk

In [ ]:
!pip install Keras

# ChatBot Code

## Importing Libraries

In [9]:
import tensorflow as tf

In [10]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to C:\Users\Muhammad
[nltk_data]     Arham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Muhammad
[nltk_data]     Arham\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Initializing Chatbot Training

In [11]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open(r'intents_edited.json').read()
intents = json.loads(data_file)

In [12]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [13]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

628 documents
23 classes ['1000-1050', '500-550', '551-600', '601-650', '650-700', '700-750', '750-800', '800-850', '850-900', '900-950', '950-1000', 'Intermediate', 'Matric', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'bye', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
660 unique lemmatized words ["'m", "'s", ',', '10', '1000', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1018', '1019', '1020', '1021', '1022', '1023', '1024', '1025', '1026', '1027', '1028', '1029', '1030', '1031', '1032', '1033', '1034', '1035', '1036', '1037', '1038', '1039', '1040', '1041', '1042', '1043', '1044', '1045', '1046', '1047', '1048', '1049', '1050', '10th', '12', '12th', '500', '501', '502', '503', '504', '505', '506', '507', '508', '509', '510', '511', '512', '513', '514', '515', '516', '517', '518', '519', '520', '521', '522', '523', '524', '525', '526', '527'

## Building the Deep Learning Model

In [14]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-14-1a891a7a8859>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [15]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("your model created")

Epoch 1/200
126/126 [==============================] - 2s 4ms/step - loss: 3.0416 - accuracy: 0.0711
Epoch 2/200
126/126 [==============================] - 0s 4ms/step - loss: 2.8362 - accuracy: 0.0669
Epoch 3/200
126/126 [==============================] - 0s 4ms/step - loss: 2.8009 - accuracy: 0.0778
Epoch 4/200
126/126 [==============================] - 1s 5ms/step - loss: 2.7405 - accuracy: 0.0751
Epoch 5/200
126/126 [==============================] - 1s 4ms/step - loss: 2.6895 - accuracy: 0.0980
Epoch 6/200
126/126 [==============================] - 1s 5ms/step - loss: 2.6266 - accuracy: 0.0998
Epoch 7/200
126/126 [==============================] - 1s 5ms/step - loss: 2.5846 - accuracy: 0.1147
Epoch 8/200
126/126 [==============================] - 1s 6ms/step - loss: 2.6026 - accuracy: 0.1033
Epoch 9/200
126/126 [==============================] - 1s 5ms/step - loss: 2.5496 - accuracy: 0.1382
Epoch 10/200
126/126 [==============================] - 1s 6ms/step - loss: 2.5232 - accura

126/126 [==============================] - 0s 4ms/step - loss: 0.1914 - accuracy: 0.9295: 0s - loss: 0.2293 - ac
Epoch 81/200
126/126 [==============================] - 0s 4ms/step - loss: 0.1485 - accuracy: 0.9563
Epoch 82/200
126/126 [==============================] - 0s 4ms/step - loss: 0.1579 - accuracy: 0.9464
Epoch 83/200
126/126 [==============================] - 0s 4ms/step - loss: 0.1088 - accuracy: 0.9628
Epoch 84/200
126/126 [==============================] - 0s 4ms/step - loss: 0.1082 - accuracy: 0.9637
Epoch 85/200
126/126 [==============================] - 1s 4ms/step - loss: 0.0928 - accuracy: 0.9663
Epoch 86/200
126/126 [==============================] - 1s 4ms/step - loss: 0.0781 - accuracy: 0.9679
Epoch 87/200
126/126 [==============================] - 1s 4ms/step - loss: 0.1173 - accuracy: 0.9588
Epoch 88/200
126/126 [==============================] - 0s 4ms/step - loss: 0.0985 - accuracy: 0.9645
Epoch 89/200
126/126 [==============================] - 1s 4ms/step - l

126/126 [==============================] - 0s 4ms/step - loss: 0.0553 - accuracy: 0.9743
Epoch 160/200
126/126 [==============================] - 0s 3ms/step - loss: 0.1249 - accuracy: 0.9567
Epoch 161/200
126/126 [==============================] - 0s 3ms/step - loss: 0.0660 - accuracy: 0.9811
Epoch 162/200
126/126 [==============================] - 0s 3ms/step - loss: 0.0569 - accuracy: 0.9835
Epoch 163/200
126/126 [==============================] - 0s 3ms/step - loss: 0.0526 - accuracy: 0.9776
Epoch 164/200
126/126 [==============================] - 0s 3ms/step - loss: 0.0403 - accuracy: 0.9855
Epoch 165/200
126/126 [==============================] - 0s 3ms/step - loss: 0.0534 - accuracy: 0.9846
Epoch 166/200
126/126 [==============================] - 0s 3ms/step - loss: 0.0692 - accuracy: 0.9678
Epoch 167/200
126/126 [==============================] - 0s 3ms/step - loss: 0.0510 - accuracy: 0.9871
Epoch 168/200
126/126 [==============================] - 0s 3ms/step - loss: 0.0371 - a

## Building Chatbot GUI

In [16]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open(r'intents_edited.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [17]:

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

## Creating GUI with tkinter

In [ ]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("UNIVERSITY ADMISSIONS ASSISTANT")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="10", width="60", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=3,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create Button to send message
# SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
#                     bd=0, bg="#23a828", activebackground="#23a828",fg='#874113',
#                     command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="50", height="3", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

##### 

--------------------------------------------------------